# Extraction et mise à jour des données pour le S&P/TSX Composite Index

Ce notebook récupère depuis Wikipedia la liste des tickers des entreprises composant l'indice S&P/TSX Composite (tableau à l'index 3). Pour chaque ticker, il effectue les opérations suivantes :

1. **Vérification** :
   - Si le ticker est déjà présent dans le CSV existant, il est analysé pour déterminer si la colonne **historical** contient des données couvrant la période souhaitée (entre `start_date` et `end_date`). Pour ce faire, le script extrait la date maximale présente dans l'historique.
   - Si la date maximale est antérieure à `end_date`, le ticker est mis à jour (données manquantes à récupérer).
   - Sinon, le ticker est ignoré (les données sont considérées comme complètes).

2. **Mise à jour** :
   - Pour les tickers absents du CSV ou dont l'historique est incomplet, le script interroge yfinance (en gérant le rate limiting par retries) pour récupérer l'historique des prix entre `start_date` et `end_date` (colonnes Date, Open, High, Low, Close, Volume et Dividends si disponibles), convertit ces données en JSON (colonne **historical**), et récupère l'ensemble des attributs (chacun dans sa propre colonne).
   - Si le ticker était déjà présent, l'historique existant est fusionné avec les nouveaux enregistrements (en ajoutant uniquement ceux dont la date n'est pas déjà présente), afin de s'assurer que toutes les dates (pour les jours de bourse) entre `start_date` et `end_date` sont couvertes.

Le résultat final, une seule ligne par ticker, est sauvegardé dans le dossier **stocks** sous le nom `sp_tsx_canadien_data.csv`.

Note : Pour les tickers du TSX, le suffixe `.TO` est ajouté si nécessaire.

In [1]:
import os
import json
import time
from datetime import datetime
import pandas as pd
import yfinance as yf

# Paramètres de dates
start_date = "2020-01-01"
end_date = "2025-04-03"
date_format = "%Y-%m-%d"

# URL de Wikipedia pour le S&P/TSX Composite Index
url = "https://en.wikipedia.org/wiki/S%26P/TSX_Composite_Index"

# Lecture des tableaux de la page
try:
    tables = pd.read_html(url, flavor="lxml")
    print("Nombre de tableaux récupérés:", len(tables))
except Exception as e:
    print("Erreur lors de la lecture de la page Wikipedia:", e)
    tables = []

if tables:
    # Utiliser le tableau à l'index 3
    df = tables[3]
    print("Aperçu du tableau récupéré:")
    print(df.head())
    
    # Identifier la colonne contenant les tickers
    if 'Ticker' in df.columns:
        tickers = df['Ticker'].tolist()
    elif 'Symbol' in df.columns:
        tickers = df['Symbol'].tolist()
    else:
        print("Aucune colonne 'Ticker' ou 'Symbol' trouvée.")
        tickers = []
    
    tickers = [str(ticker).strip() for ticker in tickers if pd.notna(ticker)]
    print("Tickers récupérés:", tickers)
else:
    print("Aucun tableau n'a été récupéré. Vérifiez l'URL ou la structure de la page.")
    tickers = []

# Créer le dossier 'stocks' s'il n'existe pas
os.makedirs("stocks", exist_ok=True)

# Obtenir la date d'aujourd'hui au format AAAAMMJJ (exemple: 20250319)
today = datetime.today().strftime('%Y%m%d')

# Créer le nom de fichier en ajoutant la date avant l'extension
filename = f"sp_tsx_canadien_data_{today}.csv"

# Construire le chemin complet
output_path = os.path.join("stocks", filename)

# Charger les données existantes (si le CSV existe)
if os.path.exists(output_path):
    existing_df = pd.read_csv(output_path)
    print(f"Fichier existant trouvé ({output_path}).")
    existing_data = {row["Ticker"]: row for _, row in existing_df.iterrows()}
else:
    existing_data = {}

def get_info_with_retry(stock, ticker, max_retries=3, delay=30):
    for attempt in range(max_retries):
        try:
            info = stock.info
            return info
        except Exception as e:
            if "Too Many Requests" in str(e):
                print(f"Rate limited pour {ticker}. Attente de {delay} secondes (tentative {attempt+1}/{max_retries})...")
                time.sleep(delay)
            else:
                raise e
    return None

# Fonction de fusion de l'historique : ajoute les enregistrements manquants
def merge_historical(existing_json, new_json):
    try:
        existing_list = json.loads(existing_json) if existing_json else []
    except Exception:
        existing_list = []
    try:
        new_list = json.loads(new_json) if new_json else []
    except Exception:
        new_list = []
    # Créer un dictionnaire par date pour les enregistrements existants
    merged_dict = {record.get("Date"): record for record in existing_list if record.get("Date")}
    # Ajouter ou mettre à jour avec les enregistrements du nouveau
    for record in new_list:
        date_key = record.get("Date")
        if date_key:
            merged_dict[date_key] = record
    merged_list = list(merged_dict.values())
    try:
        merged_list = sorted(merged_list, key=lambda x: x.get("Date", ""))
    except Exception:
        pass
    return json.dumps(merged_list, default=str)

# Fonction pour vérifier si l'historique existant est complet
def is_history_complete(historical_json, start_date, end_date):
    try:
        hist_list = json.loads(historical_json) if historical_json else []
    except Exception:
        return False
    if not hist_list:
        return False
    # Extraire les dates disponibles
    try:
        dates = [datetime.strptime(record.get("Date")[:10], date_format) for record in hist_list if record.get("Date")]
    except Exception:
        return False
    if not dates:
        return False
    min_date = min(dates)
    max_date = max(dates)
    target_end = datetime.strptime(end_date, date_format)
    target_start = datetime.strptime(start_date, date_format)
    # Vérifier que la première date est antérieure ou égale à start_date et que la dernière est supérieure ou égale à end_date
    return (min_date <= target_start) and (max_date >= target_end)

# Liste pour stocker les nouvelles données (à ajouter ou à mettre à jour)
new_data = []

for ticker in tickers:
    # Vérifier si le ticker existe déjà et si l'historique est complet
    update_required = True
    if ticker in existing_data:
        existing_hist = existing_data[ticker].get("historical", "")
        if is_history_complete(existing_hist, start_date, end_date):
            print(f"Ticker {ticker} déjà présent avec historique complet. Passage au suivant.")
            update_required = False
        else:
            print(f"Ticker {ticker} présent mais historique incomplet. Mise à jour requise.")
    
    if not update_required:
        continue
    
    # Pour le TSX, ajouter le suffixe '.TO' si nécessaire
    yf_ticker = ticker if ticker.endswith('.TO') else ticker + '.TO'
    print(f"Traitement du ticker: {yf_ticker}")
    try:
        stock = yf.Ticker(yf_ticker)
        
        # Récupérer les informations avec retry
        info = get_info_with_retry(stock, yf_ticker)
        if not info:
            print(f"Aucune information pour {yf_ticker} après plusieurs tentatives. Passage au suivant.")
            continue
        
        # Récupérer l'historique des prix entre start_date et end_date
        hist = stock.history(start=start_date, end=end_date)
        if hist.empty:
            print(f"Aucune donnée historique pour {yf_ticker}")
            continue
        
        hist.reset_index(inplace=True)
        cols = ["Date", "Open", "High", "Low", "Close", "Volume"]
        if "Dividends" in hist.columns:
            cols.append("Dividends")
        hist = hist[cols]
        historical_json = json.dumps(hist.to_dict(orient="records"), default=str)
        
        # Si le ticker est déjà présent, fusionner l'historique existant avec le nouveau
        if ticker in existing_data:
            merged_history = merge_historical(existing_data[ticker].get("historical", ""), historical_json)
        else:
            merged_history = historical_json
        
        row = {}
        row["Ticker"] = ticker
        row["historical"] = merged_history
        
        # Ajouter chaque attribut récupéré par yfinance dans sa propre colonne
        for key, value in info.items():
            if not isinstance(value, (int, float, str)):
                value = str(value)
            row[key] = value
        
        new_data.append(row)
    except Exception as e:
        print(f"Erreur pour {yf_ticker}: {e}")

# Fusionner les données existantes et les nouvelles données
final_output = list(existing_data.values()) + new_data

def sanitize_row(row):
    sanitized = {}
    for k, v in row.items():
        if not isinstance(v, (int, float, str)):
            sanitized[k] = str(v)
        else:
            sanitized[k] = v
    return sanitized

final_output = [sanitize_row(row) for row in final_output]

df_final = pd.DataFrame(final_output)
df_final.to_csv(output_path, index=False)
print(f"Les données ont été exportées dans '{output_path}'")

Nombre de tableaux récupérés: 5
Aperçu du tableau récupéré:
  Ticker                     Company       Sector [9]  \
0    AAV       Advantage Energy Ltd.           Energy   
1    AEM  Agnico Eagle Mines Limited  Basic Materials   
2     AC                  Air Canada      Industrials   
3    AGI            Alamos Gold Inc.  Basic Materials   
4   ASTL     Algoma Steel Group Inc.  Basic Materials   

                           Industry [9]  
0  Oil & Gas Exploration and Production  
1                       Metals & Mining  
2                        Transportation  
3                       Metals & Mining  
4                                 Steel  
Tickers récupérés: ['AAV', 'AEM', 'AC', 'AGI', 'ASTL', 'AQN', 'ATD', 'AP.UN', 'ALA', 'AIF', 'ARX', 'ATZ', 'ACO.X', 'ATH', 'ATRL', 'ATS', 'AYA', 'BTO', 'BDGI', 'BMO', 'BNS', 'ABX', 'BHC', 'BTE', 'BCE', 'BIR', 'BDT', 'BB', 'BEI.UN', 'BBD.B', 'BLX', 'BYD', 'BAM', 'BBU.UN', 'BN', 'BIP.UN', 'BEP.UN', 'DOO', 'CAE', 'CXB', 'CCO', 'CAR.UN', 'CM', 'CNR

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AP.UN.TO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AP.UN.TO&crumb=lmCr0C7mGOX


Erreur pour AP.UN.TO: 'NoneType' object has no attribute 'update'
Traitement du ticker: ALA.TO
Traitement du ticker: AIF.TO
Traitement du ticker: ARX.TO
Traitement du ticker: ATZ.TO
Traitement du ticker: ACO.X.TO


$ACO.X.TO: possibly delisted; no timezone found


Aucune donnée historique pour ACO.X.TO
Traitement du ticker: ATH.TO
Traitement du ticker: ATRL.TO
Traitement du ticker: ATS.TO
Traitement du ticker: AYA.TO
Traitement du ticker: BTO.TO
Traitement du ticker: BDGI.TO
Traitement du ticker: BMO.TO
Traitement du ticker: BNS.TO
Traitement du ticker: ABX.TO
Traitement du ticker: BHC.TO
Traitement du ticker: BTE.TO
Traitement du ticker: BCE.TO
Traitement du ticker: BIR.TO
Traitement du ticker: BDT.TO
Traitement du ticker: BB.TO
Traitement du ticker: BEI.UN.TO


...

Aucune donnée historique pour STLC.TO
Traitement du ticker: SJ.TO
Traitement du ticker: SVI.TO
Traitement du ticker: SLF.TO
Traitement du ticker: SU.TO
Traitement du ticker: SPB.TO
Traitement du ticker: TVE.TO
Traitement du ticker: TRP.TO
Traitement du ticker: TECK.B.TO


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TECK.B.TO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TECK.B.TO&crumb=lmCr0C7mGOX


Erreur pour TECK.B.TO: 'NoneType' object has no attribute 'update'
Traitement du ticker: T.TO
Traitement du ticker: TFII.TO
Traitement du ticker: TRI.TO
Traitement du ticker: TLRY.TO
Traitement du ticker: X.TO
Traitement du ticker: TPZ.TO
Traitement du ticker: TXG.TO
Traitement du ticker: TIH.TO
Traitement du ticker: TD.TO
Traitement du ticker: TOU.TO
Traitement du ticker: TA.TO
Traitement du ticker: TCL.A.TO


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TCL.A.TO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TCL.A.TO&crumb=lmCr0C7mGOX


Erreur pour TCL.A.TO: 'NoneType' object has no attribute 'update'
Traitement du ticker: TFPM.TO
Traitement du ticker: TSU.TO
Traitement du ticker: VRN.TO
Traitement du ticker: VET.TO
Traitement du ticker: WCN.TO
Traitement du ticker: WDO.TO
Traitement du ticker: WFG.TO
Traitement du ticker: WPM.TO
Traitement du ticker: WCP.TO
Traitement du ticker: WPK.TO
Traitement du ticker: WSP.TO
Les données ont été exportées dans 'stocks/sp_tsx_canadien_data_20250422.csv'
